In [0]:
%run /Workspace/Users/ebadurrehman2012@gmail.com/Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
files=dbutils.fs.ls(f"{dataset_bookstore}/orders-raw") 
display(files)   

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "parquet")
 .option("cloudFiles.schemaLocation", "dbfs:/mnt/demo/orders_raw")
 .load(f"{dataset_bookstore}/orders-raw")
 .createOrReplaceTempView("orders_raw_temp")
)

In [0]:
%sql
create or replace temporary view orders_temp as 
(
  select *,  current_timestamp() arrival_time, input_file_name() source_name from orders_raw_temp
)

In [0]:
%sql
select * from orders_temp

In [0]:
(spark.table("orders_temp")
.writeStream
.format("delta")
.option("checkpointLocation", "dbfs:/mnt/demo/orders_bronze")
.outputMode("append")
.table("orders_bronze")
)

In [0]:
%sql
select * from orders_bronze

In [0]:
load_new_data()

In [0]:
(spark.read
 .format("json")
 .load(f"{dataset_bookstore}/customers-json")
 .createOrReplaceTempView("customers_lookup")
)

In [0]:
%sql
select * from customers_lookup

In [0]:
(
    spark.readStream
    .table("orders_bronze")
    .createOrReplaceTempView("orders_bronze_tmp")
)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW orders_enriched_tmp AS (
SELECT order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
cast(from_unixtime (order_timestamp, 'yyyy-MM-dd HH:mm:ss') AS timestamp) order_timestamp, books
FROM orders_bronze_tmp o
INNER JOIN customers_lookup c
ON o.customer_id = c.customer_id
WHERE quantity > 0)

In [0]:
(spark.table("orders_enriched_tmp")
.writeStream
.format("delta")
.option("checkpointLocation", "dbfs:/mnt/demo/orders_sliver")
.outputMode("append")
.table("orders_silver")
)

In [0]:
%sql
select * from orders_silver

In [0]:
load_new_data()

In [0]:
(
    spark.readStream
    .table("orders_silver")
    .createOrReplaceTempView("orders_silver_tmp")
)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW daily_customer_books_tmp AS (
SELECT customer_id, f_name, l_name, date_trunc("DD", order_timestamp) order_date, sum (quantity) books_count
FROM orders_silver_tmp
GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)
)

In [0]:
(spark.table("daily_customer_books_tmp")
.writeStream
.format("delta")
.outputMode("complete")
.option("checkpointLocation", "dbfs:/mnt/demo/daily_customers_books")
.trigger(availableNow=True)
.table("daily_customers_books")
)

In [0]:
%sql
select * from daily_customers_books

In [0]:
load_new_data(all=True)